On cherche à voir quels filtres fonctionnent le mieux sur notre mini corpus de phrases alignées correctement
-----

In [29]:
with open("./data/aligned/en.txt") as f_en, open("./data/aligned/fr.txt") as f_fr:
    en_sentences = [l.strip() for l in f_en]
    fr_sentences = [l.strip() for l in f_fr]
original_pairs = list(zip(en_sentences, fr_sentences))

print(original_pairs)
print(len(original_pairs))

[('Hi, my name is Pierre and i love eating apples.', "Bonjour, je m'appelle Pierre et j'adore manger des pommes."), ('Last night, i took the bus 396 to go home from the cinema, it was dark.', "Hier soir, j'ai pris le bus 396 pour rentrer du cinéma, il faisait nuit noire."), ('This artist is my favorite artist, i discovered him during a concert in Las Vegas. His album "Vanilla" is a masterpiece.', 'Cet artiste est mon artiste préféré, je l\'ai découvert lors d\'un concert à Las Vegas. Son album "Vanilla" est un chef d\'oeuvre.'), ("Here is my email address: jean.pierre@gmail.com Please don't spam me, I hate it.", "Voici mon adresse mail : jean.pierre@gmail,com ne me spamer pas s'il vous plait je déteste ça."), ('The PS3 was the best-selling console during the period 2005-2013, with the Xbox 360 slightly behind.', 'La ps3 est la console la plus vendue pendant la période 2005-2013, la xbox 360 quant à elle est légèrement derrière.'), ('Whoa!!! Where are you going like that ? I almost fell

On va générer plusieurs configurations de fichiers yaml avec les filtres opusfilter qui nous semblent intéressants

--------

In [72]:
import itertools
import yaml
import os

def generate_config(lenght_ratio: list = None, 
                    language_id: dict = None, 
                    character_score: dict = None) -> None:
    """Cette fonction génère un ou plusieurs fichiers de configurations yaml, vous devez spécifier les arguments
    que vous souhaitez utiliser :
    - lenght_ratio => seuil pour le filtre LengthRatio
    - language_id => dictionnaire pour le filtre LanguageId
    - character_score => dictionnaire pour le filtre CharacterScore
    """
    
    filters = []
    
    # Charger la configuration de base
    with open('./data/settings_yaml/config.yaml', 'r') as f:
        base_config = yaml.safe_load(f)

    # Ajouter les filtres sélectionnés
    if lenght_ratio is not None:
        filters.append(("LengthRatioFilter", lenght_ratio))
    if language_id is not None:
        filters.append(("LanguageIdFilter", [dict(language_id)]))  
    if character_score is not None:
        filters.append(("CharacterScoreFilter", [dict(character_score)]))
    
    all_combinations = itertools.product(*[f[1] for f in filters])

   
    output_dir = './data/settings_yaml/'
    os.makedirs(output_dir, exist_ok=True)

    for combination in all_combinations:
        new_config = base_config.copy()
        new_config['steps'][0]['parameters']['filters'] = []

       
        filter_str = "_".join([f"{filters[i][0]}_{combination[i]}" for i in range(len(filters))])

        # 
        for i, (filter_name, _) in enumerate(filters):
            value = combination[i]
            if filter_name == "LengthRatioFilter":
                new_config['steps'][0]['parameters']['filters'].append({
                    filter_name: {
                        "threshold": value,
                        "name": "word",
                        "unit": "word"
                    }
                })
            else:
                new_config['steps'][0]['parameters']['filters'].append({
                    filter_name: {
                        "threshold": value
                    }
                })

        ## modifications des outputs dans le fichier yaml
        new_config['steps'][0]['parameters']['outputs'] = [
            f"en_{filter_str}.filtered.gz",
            f"fr_{filter_str}.filtered.gz"
        ]

        
        config_filename = f"config_{filter_str}.yaml"
        config_path = os.path.join(output_dir, config_filename)
        with open(config_path, 'w') as f_out:
            yaml.safe_dump(new_config, f_out, default_flow_style=False)

In [ ]:
generate_config(lenght_ratio=[0.5, 1.0, 1.5])
generate_config(language_id={"language": "en"})
generate_config(lenght_ratio=[0.5, 1.0, 1.5], language_id={"language": "en"})

On va ensuite executer les fichiers yaml

-----

In [70]:
import glob
import subprocess

yaml_files = glob.glob("data/settings_yaml/config_*.yaml")

for config_path in yaml_files:
    print(f"Running opusfilter with config: {config_path}")
    subprocess.run(["opusfilter", "--overwrite", config_path])


Running opusfilter with config: data/settings_yaml/config_LengthRatioFilter_0.5.yaml


INFO:opusfilter.opusfilter:Running step 1: filter
50it [00:00, 38593.15it/s]


Running opusfilter with config: data/settings_yaml/config_LengthRatioFilter_1.5.yaml


INFO:opusfilter.opusfilter:Running step 1: filter
50it [00:00, 17769.46it/s]


Running opusfilter with config: data/settings_yaml/config_LengthRatioFilter_1.0.yaml


INFO:opusfilter.opusfilter:Running step 1: filter
50it [00:00, 49044.71it/s]


On va analyser les fichiers générés par yaml avec nos filtres et les comparés à nos phrases alignées 

--------

In [71]:
import gzip

def analyze_results(original_pairs, thresholds):
    results = {}

    # Boucle à travers les seuils
    for threshold in thresholds:
        # Construire les chemins pour les fichiers en fonction du seuil
        en_file = f"./data/filtered/en_LengthRatioFilter_{threshold}.filtered.gz"
        fr_file = f"./data/filtered/fr_LengthRatioFilter_{threshold}.filtered.gz"
        
        # Ouvrir les fichiers filtrés pour chaque seuil
        with gzip.open(en_file, "rt", encoding="utf-8") as f_en_filt, \
             gzip.open(fr_file, "rt", encoding="utf-8") as f_fr_filt:
            
            # Créer des ensembles (sets) de paires filtrées
            filtered_pairs = set(zip(
                [l.strip() for l in f_en_filt],
                [l.strip() for l in f_fr_filt]
            ))

        # Comparer les paires originales avec les paires filtrées
        removed_pairs = [pair for pair in original_pairs if pair not in filtered_pairs]

        # Enregistrer le nombre de paires supprimées pour ce seuil
        results[threshold] = len(removed_pairs)

        # Afficher les résultats
        print(f"\n=== Threshold {threshold} ===")
        print(f"{len(removed_pairs)} paires supprimées.")
        
        if removed_pairs:  # Si des paires ont été supprimées
            print("Exemples :")
            for pair in removed_pairs[:5]:  # Afficher les 5 premiers exemples
                print(f"- EN: {pair[0]}")
                print(f"  FR: {pair[1]}")
        else:
            print("Aucune paire supprimée.")

    return results


####################################
thresholds = [0.5, 1.0, 1.5]  # Exemple de seuils
####################################""
with open("data/aligned/en.txt", "r", encoding="utf-8") as f_en, \
     open("data/aligned/fr.txt", "r", encoding="utf-8") as f_fr:
    
    original_pairs = list(zip(
        [l.strip() for l in f_en],
        [l.strip() for l in f_fr]
    ))

# Analyser les résultats pour chaque filtre et seuil
results = analyze_results(original_pairs, thresholds)



=== Threshold 0.5 ===
50 paires supprimées.
Exemples :
- EN: Hi, my name is Pierre and i love eating apples.
  FR: Bonjour, je m'appelle Pierre et j'adore manger des pommes.
- EN: Last night, i took the bus 396 to go home from the cinema, it was dark.
  FR: Hier soir, j'ai pris le bus 396 pour rentrer du cinéma, il faisait nuit noire.
- EN: This artist is my favorite artist, i discovered him during a concert in Las Vegas. His album "Vanilla" is a masterpiece.
  FR: Cet artiste est mon artiste préféré, je l'ai découvert lors d'un concert à Las Vegas. Son album "Vanilla" est un chef d'oeuvre.
- EN: Here is my email address: jean.pierre@gmail.com Please don't spam me, I hate it.
  FR: Voici mon adresse mail : jean.pierre@gmail,com ne me spamer pas s'il vous plait je déteste ça.
- EN: The PS3 was the best-selling console during the period 2005-2013, with the Xbox 360 slightly behind.
  FR: La ps3 est la console la plus vendue pendant la période 2005-2013, la xbox 360 quant à elle est légè